# Packages 

In [4]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *

import joblib
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from gensim.similarities.annoy import AnnoyIndexer


from annoy import AnnoyIndex
import polars as pl
import implicit
from src.eval import model_eval

import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir, candidate_file_name
from lightgbm import LGBMRanker
from lightgbm import early_stopping
from utils import *
# try github 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Config 

In [5]:


# 'task1': 
ranker_train_data_dir = '../data/rank_train_data_v2/'
rank_model_version = 'rank_lgbm_v4'

In [6]:
debug = False

if debug:
    SAMPLE_NUM = 1000
else:
    SAMPLE_NUM = None

candidate_path = '../data/candidates/'
model_dir = '../model_training'
# train_data_dir = '.'
# test_data_dir = '.'
task = 'task1'
w2v_model_version = 'w2v_v3'
nic_model_version = 'nic'

rank_model_dir = os.path.join(model_dir, rank_model_version)
model_for_eval = True
w2v_topn=100
nic_topn=100
# PREDS_PER_SESSION = 100

# num_tree = 100



# # target locales: locales needed for task1
target_locals = ["DE", 'JP', 'UK']

# submit_file = f'submission_{task}_ALS.parquet'
num_tree = 100
w2v_model_dir = os.path.join(model_dir, w2v_model_version)
w2v_model_file = os.path.join(w2v_model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(w2v_model_dir, f"{str(num_tree)}_{model_for_eval}.index")


sub_file = f'../data/sub_files/{rank_model_version}.parque'
eval_sub_file = f'../data/sub_files/eval_{rank_model_version}.parque'

In [7]:
rank_model_dir

'../model_training/rank_lgbm_v4'

In [8]:
! mkdir {rank_model_dir}

mkdir: cannot create directory ‘../model_training/rank_lgbm_v4’: File exists


# Read data

In [9]:


test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task3.parquet'), n_rows=SAMPLE_NUM).with_columns(pl.col('prev_items').apply(str2list))


In [11]:
test_pl.head().collect()

prev_items,locale,session_id
list[str],str,i64
"[""B0BF9JMVDG"", ""B01ET9V90M""]","""ES""",4365996
"[""B09QQG85HM"", ""B09J4T4JF5""]","""ES""",4365997
"[""B09NSKDG4K"", ""B09YY6J1ZM""]","""ES""",4365998
"[""B09B7NYDJ7"", ""B09B7NYDJ7""]","""ES""",4365999
"[""B0B6J17LK4"", ""B0B6R7X6GY"", ""B07HXY5SGH""]","""ES""",4366000


# Ranker 

In [24]:
def get_additional_feature(data_pl, session_features=session_features, product_pl=product_pl):
    session_join_keys = ['session_id']
    session_fea_cols = ['mean_price', 'last_price', 'min_price', 'max_price']
    data_pl = (
        data_pl.join(session_features.select(session_join_keys+session_fea_cols+['locale']),
                                    how='left', on=session_join_keys, suffix='_str')
            .join(product_pl.select(['id', 'locale', 'price']), how='left', right_on=['id', 'locale']
                 , left_on=['next_item_prediction', 'locale_str'])
            .with_columns(
              pl.col('price').truediv(pl.col('mean_price')).alias('price_mean_norm')
              , pl.col('price').truediv(pl.col('last_price')).alias('price_last_norm')
              , pl.col('price').truediv(pl.col('min_price')).alias('price_min_norm')
              , pl.col('price').truediv(pl.col('max_price')).alias('price_max_norm')
              # , pl.when(pl.col('min_price')==pl.col('max_price')).then(1).otherwise((pl.col('price')-pl.col('min_price'))/(pl.col('min_price')-pl.col('max_price'))).alias('price_norm')
            )
            .select(
                pl.all().exclude(session_fea_cols+['locale_str', 'price'])
            )

    )
    return data_pl

## Load Model 

In [12]:
debug

False

In [13]:
if not debug:
    ranker = joblib.load(os.path.join(rank_model_dir, 'model.pkl'))
ranker

LGBMRanker(eval_at=[3], importance_type='gain', metric='ndcg',
           n_estimators=1000, objective='lambdarank')

### Importance 

In [14]:
impotant_df = pd.DataFrame(
    {
        'features': ranker.feature_name_,
        'importance': ranker.feature_importances_
    }
).sort_values('importance', ascending=False)
impotant_df

,features,importance
0,next_item_weight,3.988920e+06
7,price_mean_norm,6.407572e+04
1,next_item_weight_nfi,6.850975e+03
2,next_item_weight_co_visit,4.961293e+03
5,last_item_similarity,2.780719e+03
4,w2v_weight,1.429808e+03
6,prev_length,6.914070e+02
9,price_min_norm,5.488500e+02
10,price_max_norm,3.044230e+02
3,locale,0.000000e+00


In [76]:
def rank_rec(candidate_pl, feature_cols, ranker, topn=100):

    candidate_pl = candidate_pl.collect()
    print(f"candidate pl shape: {candidate_pl.shape}")
    inference = ranker.predict(candidate_pl[feature_cols].to_pandas())
    test_result = (candidate_pl
         .lazy()
         .with_columns(
             pl.Series(name='predict', values=inference)
         )
         .with_columns(
            pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
         )
         .sort(['session_id', 'rank'])
         .filter(pl.col('rank')<=topn)
         .groupby(['session_id'])
         .agg(
             pl.col('next_item_prediction')
             , pl.col('next_item').unique()#.arr.get(0).cast(pl.Utf8)
         ).with_columns(
             pl.col('next_item').arr.get(0).alias('next_item')
         )
    )
    return test_result


In [77]:
# target_train_candidates.head().collect()

In [78]:
eval_candidates = get_additional_feature(data_pl=eval_candidates)

In [79]:
ranker_eval_pl = rank_rec(candidate_pl=eval_candidates,
                         feature_cols=feature_cols
                         , ranker=ranker
                        # , topn=10
                         )

candidate pl shape: (64153741, 15)


In [80]:
model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))

total_sessions,mrr,recall@20,recall@100
u32,f64,f64,f64
326443,0.3335,0.4702,0.5563


In [46]:
# model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))

In [15]:
test_candidates

NameError: name 'test_candidates' is not defined

# Test inference 

In [25]:
test_candidates = get_additional_feature(data_pl=test_candidates)

In [26]:
test_candidates = test_candidates.collect()
test_candidates.shape

(74361010, 13)

In [29]:
inference = ranker.predict(test_candidates[feature_cols].to_pandas())

In [30]:
test_result = (test_candidates
     .lazy()
     .with_columns(
         pl.Series(name='predict', values=inference)
     )
     .with_columns(
        pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
     )
     .sort(['session_id', 'rank'])
     .filter(pl.col('rank')<=100)
     .groupby(['session_id'])
     .agg(
         pl.col('next_item_prediction')
     )
)
# test_result.head().collect()

In [31]:
test_pl.schema

{'prev_items': Unknown, 'locale': Utf8, 'session_id': Int64}

In [32]:
predictions = test_pl.join(test_result, how='left', on='session_id').collect()[['locale', 'next_item_prediction']].to_pandas()

In [33]:
predictions.shape

(316972, 2)

In [34]:
check_predictions(predictions, test_sessions=test_pl.collect().to_pandas(), 
                  # check_products=True, product_df=products
                 )
# Its important that the parquet file you submit is saved with pyarrow backend
if not debug:
    predictions.to_parquet(sub_file, engine='pyarrow')

# Submit result 

In [35]:
# # You can submit with aicrowd-cli, or upload manually on the challenge page.
!aicrowd submission create -c task-1-next-product-recommendation -f {sub_file}

rank_lgbm_v4.parque ━━━━━━━━━━━━━━━ 100.0% • 299.5/299.5 MB • 2.9 MB/s • 0:00:0000:0100:07
                                                                                 ╭─────────────────────────╮                                                                                 
                                                                                 │ Successfully submitted! │                                                                                 
                                                                                 ╰─────────────────────────╯                                                                                 
                                                                                       Important links                                                                                       
┌──────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────